In [0]:
'''
#Keras Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import pandas as pd 
import numpy as np
import random
import sys
import io

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    # print()
    # print('----- Generating text after Epoch: %d' % epoch)

    # start_index = random.randint(0, len(text) - maxlen - 1)
    # for diversity in [0.2, 0.5, 1.0, 1.2]:
    #     print('----- diversity:', diversity)

    #     generated = ''
    #     sentence = text[start_index: start_index + maxlen]
    #     generated += sentence
    #     print('----- Generating with seed: "' + sentence + '"')
    #     sys.stdout.write(generated)

    #     for i in range(400):
    #         x_pred = np.zeros((1, maxlen, len(chars)))
    #         for t, char in enumerate(sentence):
    #             x_pred[0, t, char_indices[char]] = 1.

    #         preds = model.predict(x_pred, verbose=0)[0]
    #         next_index = sample(preds, diversity)
    #         next_char = indices_char[next_index]

    #         sentence = sentence[1:] + next_char

    #         sys.stdout.write(next_char)
    #         sys.stdout.flush()
    #     print()
    None

In [0]:
def generate_output():
    generated = ''
    usr_input = input("Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

In [0]:
# path = get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# with io.open(path, encoding='utf-8') as f:
#     text = f.read().lower()
# print('corpus length:', len(text))

In [0]:
# text[:100]

In [11]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Freestyle/songdata.csv")
eminem = list(data[(data['artist'] == 'Eminem')]['text'])
text = ''
for i in range(len(eminem)):
  text += eminem[i].lower()
print('corpus length:', len(text))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
corpus length: 194271


In [12]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 50


In [13]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 64744


In [14]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Vectorization...


In [0]:
# build the model: a single LSTM
# print('Build model...')
# maxlen = 40
# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax'))

# optimizer = RMSprop(lr=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
from keras.models import load_model
epochs_total = 150
model = load_model('/content/drive/My Drive/Colab Notebooks/Freestyle/keras_model_'+ str(epochs_total) + '.h5')


In [0]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=75,
          callbacks=[print_callback])
epochs_total = 225
model.save('/content/drive/My Drive/Colab Notebooks/Freestyle/keras_model_'+ str(epochs_total) + '.h5')

Epoch 1/75
64744/64744 [==============================] - 45s 694us/step - loss: 0.7881
Epoch 2/75
64744/64744 [==============================] - 43s 669us/step - loss: 0.7915
Epoch 3/75
64744/64744 [==============================] - 44s 672us/step - loss: 0.7899
Epoch 4/75
64744/64744 [==============================] - 43s 667us/step - loss: 0.7871
Epoch 5/75
64744/64744 [==============================] - 44s 673us/step - loss: 0.7835
Epoch 6/75
64744/64744 [==============================] - 43s 672us/step - loss: 0.7802
Epoch 7/75
64744/64744 [==============================] - 44s 682us/step - loss: 0.7774
Epoch 8/75
64744/64744 [==============================] - 44s 673us/step - loss: 0.7796
Epoch 9/75
64744/64744 [==============================] - 44s 674us/step - loss: 0.7774
Epoch 10/75
64744/64744 [==============================] - 43s 671us/step - loss: 0.7802
Epoch 11/75
64744/64744 [==============================] - 43s 667us/step - loss: 0.7794
Epoch 12/75
64744/64744 [=====

In [0]:
Tx = 40
generate_output()

Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: hello


Here is your poem: 

hellojecsa, but little bith-briffes  
gettin' that the seun chooze my said  
tried, it take up in oh wokes gooda man  
i'm like  
  
i never the surrus on my trounda niggas of else flock nokes the mind of haid  
maybe murd  
its get up in last  
ter you be doneekes what you gonna like  
with it  
i'm altex  
  
it well the first you we goes at like to a familif, spitting  
i'm instroit, ho'do, i'm dout